In [3]:
# prapare packages
import numpy as np
import tensorflow as tf
import matplotlib as plt

In [4]:
# True data distribution for discriminator
# p_d(x)
class DataDistribution(object):
    # in: mu, sigma
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
    
    # returns N samples of sorted normal N ~ (mu, sigma)
    def sample(self, N):
        samples = np.random.normal(self.mu, self.sigma)
        samples.sort()
        return samples

In [5]:
# p_z(z)
# generator distribution
class NoiseDistribution(object):
    def __init__(self, range):
        self.range = range
    
    def sample(self, N):
        offset = np.random.random(N) * (float(self.range) / N)
        samples = np.linspace(-self.range, self.range, N) + offset
        return samples

In [6]:
def generator_layer(x, n_hidden):
    '''
    x: 
    '''
    input_shape = x.get_shape()
    w_initializer = tf.contrib.truncated_normal_initializer(stddev=0.01)
    b_initializer = tf.constant_initializer(0.0)
    
    

In [7]:
def discriminator_layer(x, n_hidden):
    '''
    x: input gaussian sample
    n_hidden: number of hidden units
    '''
    input_shape = x.get_shape()
    w_initializer = tf.contrib.variance_scaling_initializer()
    b_initializer = tf.constant_initializer(0.0)    
    
    # 1st hidden layer
    w0 = tf.get_variable('w0', [input_shape[1], n_hidden], initializer=w_initializer)
    b0 = tf.get_variable('b0', [n_hidden], initializer=b_initializer)
    h0 = tf.nn.relu(tf.matmul(x, w0) + b0)
    
    # output layer
    w1 = tf.get_variable('w1', [n_hidden, 1], initializer=w_initializer)
    b1 = tf.get_variable('b1', [1], initializer=b_initializer)
    out = tf.sigmoid(tf.matmul(h0, w1) + b1)
    
    return out